#                     Tourist Destination Recommendation System

# Importing libraries.

In [7]:
#Importing Liabrires
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Loading Datasets.

In [8]:
#importing data from csv file.
datasets = pd.read_csv("DataSets.csv");

#defining index as User.
datasets.set_index('Users', inplace=True);

#displaying the data sets
print(datasets.head(20));

                  Pokhara  Kathmandu  Surkhet  Lumbini  Dopla  Illam  Mustang  Maygdi  Rolpa
Users                                                                                       
Kumar Pulami          9.0          8      9.0      NaN    4.0    6.0      6.0     NaN    NaN
Sashwot Dhital        8.0          7      5.0      4.0    7.0    7.0      NaN     NaN    6.0
Ashim Nepal           8.0          8      3.0      NaN    7.0    8.0      NaN     5.0    NaN
Bandana Simkhada      9.0          5      5.0      NaN    8.0    NaN      NaN     7.0    7.0
Parbati Shrestha      8.0          4      7.0      6.0    NaN    NaN      6.0     NaN    8.0
Rosha Adhikari        8.0          6      NaN      5.0    9.0    NaN      8.0     6.0    NaN
Prakash Khadka        NaN          7      7.0      8.0    5.0    NaN      NaN     6.0    8.0
Ramesh Budha          5.0          8      5.0      5.0    7.0    NaN      7.0     NaN    6.0


# Selecting user who will get recommend.

In [9]:
#getting and storing all user names only to list.
userList = datasets.index.values;

#Listing the all the usernames stored in list.
print("List of User:")
print()
for user in userList:
    print(user)
    
#Asking to select user who wil get the recommendation.
print()
print("******************************************************************************************")
selectedUser = input ("Enter the UserName to get recommendation: ");
print("******************************************************************************************")
print()

#Storing all the user names expect selected user in order to find similarites.
remainingUser = np.delete(userList, np.where(userList == selectedUser))

List of User:

Kumar Pulami
Sashwot Dhital
Ashim Nepal
Bandana Simkhada
Parbati Shrestha
Rosha Adhikari
Prakash Khadka
Ramesh Budha

******************************************************************************************
Enter the UserName to get recommendation: Ramesh Budha
******************************************************************************************



# Helping Methods.

In [10]:
# Method to calculate Pearson Correlation Coefficient bewteen selected two users.
def calculatePearson(x,y):
    #implementing Pearson Correlation Coefficient
    upperPortion = sum((x-np.mean(x))*(y-np.mean(y)))
    lowerPortion = pow(sum(pow(x-np.mean(x),2)),0.5)*pow(sum(pow(y-np.mean(y),2)),0.5)
    
    return upperPortion/lowerPortion    
    

# Method to return the sum of all user Pearson Correlation Coefficient.
def sumPearsonValue(pearsonlist):
    sum=0
    for value in pearsonlist:
        sum = sum + value[1]
    return sum


# Method to recommend the tourist destination.
# Parameters:
# 1. List of destination which is not rated by selected users. 
# 2. List of each user's pearson value respective to selected users.
# 3. List of user expect selected user.
def getRecommendation(destinationList, calculatedPearsonValues, userList):
    # Initalize the empty list to store destination name and its predicted rating value.
    recommendation = [];
    for destination in destinationList: # Iterating  destination in destinationList
        pearsonValues = calculatedPearsonValues;
        sumPearson = 0.0;
        sumPearsonRating = 0.0;
        predictedRating = 0.0;
        for user in userList: # Iterating users in userList
            pearsonValue = 0
            if pd.notnull(datasets.loc[user][destination]): # if current user has rated on current destination,
                                                            # get pearson value, multiply with his destiantion rating, 
                                                            # and adding it with other user value .
                for pearson in pearsonValues:
                    if user == pearson[0]: 
                        if (pearson[1] >= 0): 
                            pearsonValue = pearson[1];
                            sumPearsonRating = sumPearsonRating + (pearsonValue * datasets.loc[user][destination]);
                        else:
                            pearsonValue = 0
                            pearsonValues.remove(pearson)   # getting the pearson value of current user respective to selected user.
                        
                # Multiplying the current user rating of current destination with current user pearson value, 
                #adding it with other value.       
                #sumPearsonRating = sumPearsonRating + (pearsonValue * datasets.loc[user][destination]); 
                
            elif pd.isnull(datasets.loc[user][destination]): 
                # else removing the current pearson value from the list to avoid adding current pearson value.
                for pearson in pearsonValues:
                    if user == pearson[0]:
                        pearsonValues.remove(pearson);
        sumPearson = sumPearsonValue(pearsonValues);# Calling method to sum all the user pearson value who rated the destination.
        if (sumPearson != 0):
            predictedRating = sumPearsonRating/sumPearson; # Prediction the user rating on current destination.
            recommendation.append([destination, predictedRating]); # Saving destination with predicted rating value.
    return recommendation;

# Calculating Pearson Correlation Coefficient.

In [11]:
destinationList = datasets.columns; # List to store all destination names.
ratedDestination = []; # List to store rated destination by selected user.
nonratedDestination =[]; # List to store destination that isnot rated by select user
                            # and will be used to recommend.

# Seperating the destination based on rate and not rated by selected user.
for destination in destinationList:
    if pd.notnull(datasets.loc[selectedUser][destination]): # if selected user has rated the current destination,  
        ratedDestination.append(destination);               # adding this destination to ratedDestination list.
        
    else:                                         # if selected user hasn't rated the current destination, adding 
        nonratedDestination.append(destination);  # adding this destination to nonratedDestination list.
            
        
selectUserRating = []; # List to store rating of selected user.
comparingUserRating = []; # List to store rating of user whom is going to calcute the pearson value with current user.
pearsonValueList = []; # list to store username and their pearson value.

for user in remainingUser: # Iterating the all user expect selected user         
    for destination in ratedDestination: # Iterating the destination which is rated by the selected user.
        if pd.notnull(datasets.loc[user][destination]): # if iterated user has rated the iterated rated destination by selectd user
            comparingUserRating.append(datasets.loc[user][destination]); # Adding rating of destination rated by iterated user to comparingUserRating list.
            selectUserRating.append(datasets.loc[selectedUser][destination]); # Adding rating of destination rated by iterated user to selectUserRating list.
    value = calculatePearson(selectUserRating, comparingUserRating);# calculating the pearson value passing the rating of iterated user and selected user
    pearsonValueList.append([user, value]); # adding value to pearsonValue list.
    selectUserRating.clear(); # clearing the value of selectUserRating list.
    comparingUserRating.clear(); # clearing the value of comparingUserRating list.

# Calculating Recommendation.

In [12]:
# Calling getRecommendation method.
recommendation = getRecommendation(nonratedDestination, pearsonValueList, remainingUser)
print()
print("******************************************************************************************")
print("Recommended List Of Destination:")
print("******************************************************************************************")
 # Iterating the destination
for destination in recommendation:
    if destination[1] >= 6: # if the predected rating of iterated destination is greater than  or equal to 6, show destination as recommendaiton.
        print(destination[0])
        print()
print(recommendation)


******************************************************************************************
Recommended List Of Destination:
******************************************************************************************
Illam

[['Illam', 7.544410277945063], ['Maygdi', 5.0]]
